## Installation
`pip install crewai`

`pip install langchain`

`pip install duckduckgo-search`

`ollama pull openhermes`

## LLM Chain
- Download Ollama `curl -fsSL https://ollama.com/install.sh | sh`
- Fetch a model via `ollama pull llama2`

In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [2]:
llm.invoke("how can langsmith help with testing?")

"\nLangsmith is a tool that can assist with testing by providing features such as:\n\n1. Code generation: Langsmith can generate test code automatically based on the API documentation of the library or framework being tested, saving time and effort compared to writing tests manually.\n2. Test case generation: Langsmith can generate test cases based on the API documentation of the library or framework being tested, ensuring that all possible scenarios are covered.\n3. Test suite optimization: Langsmith can optimize test suites by identifying redundant or unnecessary tests and removing them, resulting in a more efficient testing process.\n4. Defect prediction: Langsmith can analyze the API documentation of a library or framework to predict potential defects that may arise during testing, allowing developers to address these issues proactively.\n5. Testing for edge cases: Langsmith can generate test cases that cover edge cases, such as boundary value analysis, to ensure that the library o

Prompt is any form of text, question, information, or coding that communicates to AI what response you're looking for.

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

chain = prompt | llm 

In [4]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Lingua Smith, or Langsmith for short, is a fantastic tool to help with testing! Here are some ways in which Langsmith can assist with testing:\n\n1. Automated Translation: With Langsmith's automated translation capabilities, you can quickly test your content in multiple languages without the need for manual translation. This saves time and resources while ensuring that your documentation is accurate and consistent across different languages.\n2. Grammar and Spell Checking: Langsmith's grammar and spell checking features help identify errors and inconsistencies in your content, ensuring that your documentation is free of mistakes and easy to read. This can significantly improve the overall quality of your technical documentation.\n3. Style Guidance: Langsmith's style guidance feature helps enforce consistent writing styles throughout your documentation, which can be especially useful when working with multiple authors o

Output_parser converts the chat message to a string. In this case, there is no effect because the model is ChatModel.

In [5]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

In [6]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Lingua Smith, or Langsmith for short, is an excellent tool for assisting with testing. Here are some ways in which Langsmith can help:\n\n1. Automated Translation: With Langsmith's automated translation capabilities, you can easily test your documentation in multiple languages without the need for manual translations. This can save a significant amount of time and resources, allowing you to focus on other important tasks.\n2. Consistency Checking: Langsmith's advanced algorithms can help ensure that your documentation is consistent across all languages and regions. This can help prevent errors and inconsistencies that could otherwise impact the quality of your documentation.\n3. Grammar and Spell Checking: Langsmith's grammar and spell checking capabilities can help identify and correct errors in your documentation, ensuring that it is error-free and professional-looking.\n4. Terminology Management: With Langsmith, you

## Agents

Import crewAI and Ollama and initialize LLM

In [7]:
from crewai import Agent, Task, Crew
from langchain.llms import Ollama
ollama_llm = Ollama(model="openhermes")

Import and initialize DuckDuckGo


In [8]:
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

Configure the first agent. I am going with the "Researcher".

In [9]:
researcher = Agent(
  role='Researcher',
  goal='Search the internet for the information requested',
  backstory="""
  You are a researcher. Using the information in the task, you find out some of the most popular facts about the topic along with some of the trending aspects.
  You provide a lot of information thereby allowing a choice in the content selected for the final blog
  """,
  verbose=True,            # want to see the thinking behind
  allow_delegation=False,  # Not allowed to ask any of the other roles
  tools=[search_tool],     # Is allowed to use the following tools to conduct research
  llm=ollama_llm           # local model
)


Configure the second agent. I am going to call this the "Technical Content Creator".

In [10]:
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on a set of information provided by the researcher.',
  backstory="""You are a writer known for your humorous but informative way of explaining. 
  You transform complex concepts into compelling narratives.""",
  verbose=True,            # want to see the thinking behind
  allow_delegation=True,   # can ask the "researcher" for more information
  llm=ollama_llm           # using the local model
)
# Define the task that  needs to be done by the "Researcher". Call this "Task1"
task1 = Task(
  description="Research about open source LLMs vs closed source LLMs.",
  expected_output='Your final answer MUST be a full analysis report',
  agent=researcher
)


Define the task that needs to be done by the "Technical Content Creator". Call this "Task2"

In [11]:
task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant facts and differences between open-source LLMs and closed-source LLMs.""",
  expected_output="""Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, and avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 4 paragraphs.
  The target word count for the blog post should be between 1,500 and 2,500 words, with a sweet spot at around 2,450 words.""",
  agent=writer
)

Instantiate the "crew" with a sequential process

In [12]:
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 for different logging levels
)

Get your crew to start work

In [13]:
result = crew.kickoff()

 [DEBUG]: == Working Agent: Researcher
 [INFO]: == Starting Task: Research about open source LLMs vs closed source LLMs.


> Entering new CrewAgentExecutor chain...
I should start by gathering information about open source LLMs and closed source LLMs to compare their features, advantages, and disadvantages.

Action: duckduckgo_search
Action Input: {"q": "features of open source LLMs"} 

I encountered an error while trying to use the tool. This was the error: DuckDuckGoSearchRun._run() got an unexpected keyword argument 'q'.
 Tool duckduckgo_search accepts these inputs: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.

Thought: I should gather information about open source LLMs.
Action: duckduckgo_search
Action Input: {"q": "features of open source LLMs"} 

I encountered an error while trying to use the tool. This was the error: DuckDuckGoSearchRun._run() got an unexpected keyword argument 'q'.
 Tool d

2024-05-31 15:29:40,980 - 6313242624 - __init__.py-__init__:369 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/urllib3/connectionpool.py", line 793, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/langchain/lib/python3.11/http/client.py", line 1386, in getresponse
    response.begin()
  File "/opt/anaconda3/envs/langchain/lib/python3.11/http/client.py", line 325, in begin
    version, status, reason = self._read_status()
                

In [14]:
print(result)

<|im_sep|>

Paragraph 1:
When it comes to Large Language Models (LLMs), there are two primary options available in the market - open-source and closed-source. Both have their unique characteristics, benefits, and drawbacks that make them suitable for different use cases. Open-source LLMs prioritize transparency, collaboration, customization, and cost-effectiveness by making their source code, data, and training process freely available to anyone who wants to contribute or adapt the model. This approach is ideal for projects where teamwork, flexibility, and cost control are crucial. However, open-source models might suffer from limitations in performance and potential security risks due to their public nature.

Paragraph 2:
On the other hand, closed-source LLMs provide better performance, security, and intellectual property protection by keeping their underlying code, data, and training process confidential. These models are designed for organizations that require a higher level of secu